# ReAct

AutoGen supports different LLM prompting and reasoning strategies, such as ReAct, Reflection/Self-Critique, and more. This page demonstrates how to realize ReAct ([Yao et al., 2022](https://arxiv.org/abs/2210.03629)) with AutoGen. 

In [7]:
# %pip install "pyautogen>=0.2.18"

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
)
%env TAVILY_API_KEY="YOUR_API_KEY"

### Define and add tools/actions you want LLM agent(s) to use

In [ ]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)
search_tool = TavilySearchResults()


def generate_llm_config(tool):
    # Define the function schema based on the tool's args_schema
    function_schema = {
        "name": tool.name.lower().replace(" ", "_"),
        "description": tool.description,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }

    if tool.args is not None:
        function_schema["parameters"]["properties"] = tool.args

    return function_schema


# Construct the llm_config
llm_config = {
    # Generate functions config for the Tool
    "functions": [
        generate_llm_config(search_tool),
    ],
    "config_list": config_list,  # Assuming you have this defined elsewhere
    "timeout": 120,
}

### Construct your ReAct prompt
Here we are constructing a general ReAct prompt and a custom message function `react_prompt_message` based on the ReAct prompt.

In [ ]:
# NOTE: this ReAct prompt is adapted from Langchain's ReAct agent: https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/react/agent.py#L79
ReAct_prompt = """
Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
"""

# Define the ReAct prompt message. Assuming a "question" field is present in the context


def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

### Construct agents and initiate agent chats

In [17]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        # "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

# Register the tool and start the conversation
user_proxy.register_function(
    function_map={
        search_tool.name: search_tool._run,
    }
)

assistant = autogen.AssistantAgent(
    name="Assistant",
    system_message="Only use the tools you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

user_proxy.initiate_chat(
    assistant,
    message=react_prompt_message,
    question="What is the result of super bowl 2024?",
)

user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: What is the result of super bowl 2024?




--------------------------------------------------------------------------------
Assistant (to user_proxy):

Thought: I need to find the most current information on the result of the Super Bowl 2024.
Action: Use the search tool provided to find the result.
Action Input: query = "Super Bowl 2024 result"

***** Suggested function Call: tavily_search_results_json *****
Arguments: 
{"query":"Super Bowl 2024 result"}
***************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION tavily_search_results_json...
user_proxy (to Assistant):

***** Response from calling function "tavily_search_results_json" *****
[{'url': 'https://www.sportingnews.com/us/nfl/news/super-bowl-2024-live-score-49ers-chiefs-results-highlights/0c440aa7145b809ed174d8ff', 'content': 'Super Bowl start time\nSuper Bowl 58 between the Chiefs and 49ers is set to kick off at 6:30 p.m. ET (3:30 p.m. local time) in La

ChatResult(chat_id=None, chat_history=[{'content': '\nAnswer the following questions as best you can. You have access to tools provided.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take\nAction Input: the input to the action\nObservation: the result of the action\n... (this process can repeat multiple times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\nQuestion: What is the result of super bowl 2024?\n', 'role': 'assistant'}, {'content': 'Thought: I need to find the most current information on the result of the Super Bowl 2024.\nAction: Use the search tool provided to find the result.\nAction Input: query = "Super Bowl 2024 result"\n', 'function_call': {'arguments': '{"query":"Super Bowl 2024 result"}', 'name': 'tavily_search_results_json'}, 'role': 'assistant'}, {'content': '[{\'url\': \'https://www.sportingnews.c

### ReAct with memory module enabled via teachability

In [18]:
from autogen.agentchat.contrib.capabilities import teachability

# Instantiate the Teachability capability. Its parameters are all optional.
teachability = teachability.Teachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./tmp/notebook/teachability_db",
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
teachability.add_to_agent(assistant)

user_proxy.initiate_chat(
    assistant,
    message=react_prompt_message,
    question="What is the result of super bowl 2024?",
)


CLEARING MEMORY
user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: What is the result of super bowl 2024?


--------------------------------------------------------------------------------
Assistant (to user_proxy):

Thought: To find the result of Super Bowl 2024, I need to search for the most recent information available since the event is in the future as of my knowledge cutoff, and the result would only be known after the event takes place.

Action: Use the provided search function to find the result of Super Bowl 2024.
Action Input: { query

/Users/qingyunwu/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 33.3MiB/s]


Assistant (to user_proxy):

Thought: The search results have provided detailed information on Super Bowl 2024, indicating that the Kansas City Chiefs won the Super Bowl in an overtime victory against the San Francisco 49ers with a final score of 25-22. This marks their third Super Bowl win in five years and their second consecutive title.

Final Answer: The Kansas City Chiefs won Super Bowl 2024 with a 25-22 victory over the San Francisco 49ers in overtime.

--------------------------------------------------------------------------------
user_proxy (to Assistant):



--------------------------------------------------------------------------------
Assistant (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': '\nAnswer the following questions as best you can. You have access to tools provided.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take\nAction Input: the input to the action\nObservation: the result of the action\n... (this process can repeat multiple times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\nQuestion: What is the result of super bowl 2024?\n', 'role': 'assistant'}, {'content': 'Thought: To find the result of Super Bowl 2024, I need to search for the most recent information available since the event is in the future as of my knowledge cutoff, and the result would only be known after the event takes place.\n\nAction: Use the provided search function to find the result of Super Bowl 2024.\nAction Input: { query: "Super Bowl 2024 result" }', 'function_call':

### Let's now ask the same question again to see if the assistant has remembered this.

In [20]:
user_proxy.initiate_chat(
    assistant,
    message=react_prompt_message,
    question="What is the result of super bowl 2024?",
    max_turns=1,
)

user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: What is the result of super bowl 2024?


--------------------------------------------------------------------------------
Assistant (to user_proxy):

Thought: The user has given me a memory that directly answers the question about the result of Super Bowl 2024.
Action: I will recall the information from the memory provided.
Final Answer: The Kansas City Chiefs won Super Bowl 2024 with a 25-22 overtime victory against the San Francisco 49ers.

-----------------------------------------------------

ChatResult(chat_id=None, chat_history=[{'content': '\nAnswer the following questions as best you can. You have access to tools provided.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take\nAction Input: the input to the action\nObservation: the result of the action\n... (this process can repeat multiple times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\nQuestion: What is the result of super bowl 2024?\n', 'role': 'assistant'}, {'content': 'Thought: The user has given me a memory that directly answers the question about the result of Super Bowl 2024.\nAction: I will recall the information from the memory provided.\nFinal Answer: The Kansas City Chiefs won Super Bowl 2024 with a 25-22 overtime victory against the San Francisco 49ers.', 'role': 'user'}], summary='Thought: The user has given me a memory that directly an